In [ ]:
import ollama

# nomic-embed-text has a dimention of 768
print(ollama.embeddings(model='nomic-embed-text', prompt='hello i am a chatbot')['embedding'])

In [ ]:
import ollama
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

def plot_embeddings_3d(words, embedding_model='nomic-embed-text'):
    # Dictionary to store embeddings
    vector_store = {}
    
    # Generate embeddings using ollama
    for word in words:
        embedding = ollama.embeddings(model=embedding_model, prompt=word)
        vector_store[word] = embedding['embedding']  # Assuming the embedding is stored under 'embedding' key
    
    # Convert embeddings to a numpy array for PCA
    embeddings = np.array([vector_store[word] for word in words])
    
    # Apply PCA to reduce to 3 dimensions
    pca = PCA(n_components=3)
    reduced_embeddings = pca.fit_transform(embeddings)
    
    # Plot the 3D graph
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot each word in the 3D space
    for i, word in enumerate(words):
        ax.scatter(reduced_embeddings[i, 0], reduced_embeddings[i, 1], reduced_embeddings[i, 2], label=word)
        ax.text(reduced_embeddings[i, 0], reduced_embeddings[i, 1], reduced_embeddings[i, 2], word)
    
    # Set labels for the axes
    ax.set_xlabel('PCA1')
    ax.set_ylabel('PCA2')
    ax.set_zlabel('PCA3')
    
    # Show the plot
    plt.show()

In [ ]:
words = ['Cease', 'terminate', 'desist', 'Begin', 'Originate', 
         'Abundant', 'Plentiful', 'Scarce', 'Limited', 'Ample', 'Bountiful', 'Rare']

In [ ]:
plot_embeddings_3d(words)

In [ ]:
import weaviate
import weaviate.classes.config as wc
import weaviate.classes as wvc

client = weaviate.connect_to_local()

if not client.collections.exists(name = 'dev_day_demo'):
    client.collections.create(name = 'dev_day_demo')

collection = client.collections.get('dev_day_demo')

In [ ]:
weaviate_docs = []

for index, word in enumerate(words):
    weaviate_doc = wvc.data.DataObject(
        properties={
            'message': word,
        },
        vector=list(ollama.embeddings(model='nomic-embed-text', prompt=word)['embedding'])
    )
    weaviate_docs.append(weaviate_doc)
      

In [ ]:
collection.data.insert_many(weaviate_docs)

In [ ]:
retrieved_docs = collection.query.near_vector(
    near_vector=list(ollama.embeddings(model='nomic-embed-text', prompt='more than enough')['embedding']),
    limit=2,
    return_metadata=wvc.query.MetadataQuery(certainty=True)
)

[(o.properties['message'], o.metadata.certainty) for o in retrieved_docs.objects]


In [ ]:
retrieved_docs = collection.query.near_vector(
    near_vector=list(ollama.embeddings(model='nomic-embed-text', prompt='in short supply')['embedding']),
    limit=2,
    return_metadata=wvc.query.MetadataQuery(certainty=True)
)

[(o.properties['message'], o.metadata.certainty) for o in retrieved_docs.objects]